In [ ]:
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
import tkinter.filedialog as tkf
import tifffile

In [ ]:
def gaussian(height, center_x, center_y, width_x, width_y):
    """Returns a gaussian function with the given parameters"""
    width_x = float(width_x)
    width_y = float(width_y)
    return lambda x,y: height*np.exp(
                -(((center_x-x)/width_x)**2+((center_y-y)/width_y)**2)/2)

def moments(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution by calculating its
    moments """
    total = data.sum()
    X, Y = np.indices(data.shape) # row, col
    x = (X*data).sum()/total # row
    y = (Y*data).sum()/total # col
    col = data[:, int(y)]
    width_x = np.sqrt(np.abs((np.arange(col.size)-y)**2*col).sum()/col.sum()) # row
    row = data[int(x), :]
    width_y = np.sqrt(np.abs((np.arange(row.size)-x)**2*row).sum()/row.sum()) # col
    height = data.max()
    return height, x, y, width_x, width_y

def fitgaussian(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution found by a fit"""
    params = moments(data)
    errorfunction = lambda p: np.ravel(gaussian(*p)(*np.indices(data.shape)) -
                                 data)
    p, success = optimize.leastsq(errorfunction, params)
    return p

In [ ]:
raw_adr = tkf.askopenfilename()
print(raw_adr)

In [ ]:
f_stack = tifffile.imread(raw_adr)
stack_shape = f_stack.shape
print(f_stack.shape)
print(f_stack.dtype)

In [ ]:
pacbed = np.sum(f_stack, axis=(0, 1))
pacbed = pacbed / np.max(pacbed)
print(pacbed.shape)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(pacbed, cmap="jet")
ax.axis("off")
plt.show()

In [ ]:
grad = np.gradient(pacbed)
grad_map = grad[0]**2 + grad[1] **2
grad_map = grad_map / np.max(grad_map)
print(np.min(grad_map), np.max(grad_map))

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(grad_map, cmap="jet")
ax.axis("off")
plt.show()

In [ ]:
Y, X = np.indices(pacbed.shape)
com_y = np.sum(pacbed * Y) / np.sum(pacbed)
com_x = np.sum(pacbed * X) / np.sum(pacbed)
print(com_y, com_x)
print(np.around(com_y), np.around(com_x))

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(pacbed, cmap="jet")
ax[0].axis("off")
ax[0].scatter(com_x, com_y, s=15, c="y")
ax[1].imshow(grad_map, cmap="jet")
ax[1].axis("off")
ax[1].scatter(com_x, com_y, s=15, c="y")
plt.show()

In [ ]:
(_, center_y, center_x, _, _) = fitgaussian(pacbed)
print(center_y, center_x)
print(np.around(center_y), np.around(center_x))

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(pacbed, cmap="jet")
ax[0].axis("off")
ax[0].scatter(center_x, center_y, s=15, c="y")
ax[1].imshow(grad_map, cmap="jet")
ax[1].axis("off")
ax[1].scatter(center_x, center_y, s=15, c="y")
plt.show()

In [ ]:
ct = [com_y, com_x]
ct_ind = np.around([com_y, com_x]).astype(np.int)

max_ind = np.unravel_index(np.argmax(grad_map, axis=None), grad_map.shape)
least_R = ((max_ind[0]-ct_ind[0])**2 + (max_ind[1]-ct_ind[1])**2)**(1/2)
print(least_R)

min_edge = int(np.min(ct_ind))
print(min_edge)

cropped_size = int(least_R)
c_ct = [cropped_size, cropped_size]
print(c_ct)

In [ ]:
center_aligned_stack = f_stack[:, :, ct_ind[0]-cropped_size:ct_ind[0]+cropped_size+1, ct_ind[1]-cropped_size:ct_ind[1]+cropped_size+1].copy()
print(center_aligned_stack.shape)

In [ ]:
c_pacbed = np.sum(center_aligned_stack, axis=(0, 1))
c_pacbed = c_pacbed / np.max(c_pacbed)
print(c_pacbed.shape)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(c_pacbed, cmap="jet")
ax.scatter(c_ct[0], c_ct[1], s=15, c="y")
ax.axis("off")
plt.show()

In [ ]:
th_grad = grad_map > np.percentile(grad_map, 95)
th_grad = th_grad.astype(np.float)
print(np.min(th_grad), np.max(th_grad))
print(th_grad.shape)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(th_grad, cmap="jet")
ax.axis("off")
plt.show()

In [ ]:
def twod_circle(cy, cx, R):
    full_circle = lambda x, y: (cy-y)**2+(cx-x)**2 <= R**2
    float_circle = lambda x, y: full_circle(x, y).astype(np.float)
    grad_circle = lambda x, y: np.gradient(float_circle(x, y))[0]**2 + np.gradient(float_circle(x, y))[1]**2
    
    return grad_circle

def fit_circle(data, fparam):
    err_func = lambda p: np.ravel(twod_circle(*p)(*np.indices(data.shape)) - data)
    p, success = optimize.leastsq(err_func, fparam)
    
    return p

In [ ]:
fcy, fcx = int(th_grad.shape[0]/2), int(th_grad.shape[1]/2)
print(fcy, fcx)

max_ind = np.unravel_index(np.argmax(grad_map, axis=None), grad_map.shape)
fR = ((max_ind[0]-fcy)**2 + (max_ind[1]-fcx)**2)**(1/2)
print(fR)

f_param = [fcy, fcx, fR]
print(f_param)

In [ ]:
test = twod_circle(*f_param)(*np.indices(grad_map.shape))
print(np.min(test), np.max(test))

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(test, cmap="jet")
ax.axis("off")
plt.show()

In [ ]:
params = fit_circle(pacbed, f_param)
print(params)